In [1]:
from chemtbd.io import Agilent
from scipy.stats import linregress
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
agi = Agilent.from_root('data/test3')

__matchlib2area__ matches lib dataframe to fid or tic dataframes for all data sets in a folder containing .D files generated from Agilent. The input is two dataframes from agi, agi_results_lib and agi.results_tic. The later can also be agi.results_fid

example:
```
matched_df = matchlib2area_all(agi.results_lib,agi.results_tic)
```

In [65]:
libdf = (agi.results_lib)
areadf = (agi.results_tic)

def find_match(x, Y):
    ''' find index of argmin lambda(x,Y)
    if .idxmin()>threshold return NA
    '''
    threshold = 0.001
    score = lambda y: (x - y)**2
    value = Y.apply(score)
    print(value)
#     if value>threshold:
#         pass
#         return_value = np.nan
#     else:
    return_value = Y.apply(score).idxmin()
    return return_value

def rt_match(lib_row, rt):
    ''' find closest rt
    
    '''
    x, i = lib_row.rt, lib_row.name
    return find_match(x, rt[i:])

def matchiter(libdf,areadf):
    '''match area on rt from single df
    len(libdf)<len(areadf) then switch
    '''
    xi = libdf.apply(rt_match, rt=areadf.rt.sort_values(), axis=1)
    libdf['area'] = areadf.area[xi].values
    return libdf.set_index('key')
    
def matchlib2area(libdf,areadf):
    '''match area from many datafrmas
    '''
    libdf_grouped = (libdf.groupby(libdf.index))
    areadf_grouped = (areadf.groupby(areadf.index))
    returndf = libdf_grouped.apply(lambda x: matchiter(x.reset_index(),areadf_grouped.get_group(x.name).reset_index()))    
    return returndf

matcheddf = matchlib2area(libdf,areadf)
matcheddf.reset_index().loc[matcheddf.reset_index()['key'] == 'FA03.D']

0     9.000000e-08
1     2.422070e+00
2     5.054853e+00
3     8.319186e+00
4     1.218219e+01
5     1.604243e+01
6     1.650228e+01
7     2.090593e+01
8     2.143042e+01
9     2.646382e+01
10    2.764972e+01
11    3.359709e+01
12    3.487257e+01
13    3.940450e+01
14    4.084872e+01
15    4.247520e+01
16    5.373330e+01
17    5.432132e+01
18    5.491255e+01
19    5.644968e+01
20    5.826727e+01
21    6.611804e+01
22    6.921075e+01
23    7.220411e+01
24    7.396516e+01
25    8.172702e+01
26    8.784001e+01
27    8.945944e+01
Name: rt, dtype: float64
1     1.000000e-08
2     4.787256e-01
3     1.763318e+00
4     3.739969e+00
5     5.997111e+00
6     6.279535e+00
7     9.095653e+00
8     9.442714e+00
9     1.287303e+01
10    1.370406e+01
11    1.797675e+01
12    1.891293e+01
13    2.228690e+01
14    2.337626e+01
15    2.461053e+01
16    3.333792e+01
17    3.380143e+01
18    3.426815e+01
19    3.548466e+01
20    3.692871e+01
21    4.322931e+01
22    4.573682e+01
23    4.817609e+01
24    

Name: rt, dtype: float64
3    9.568752e-01
4    1.297440e-01
5    2.226064e-02
6    4.000000e-08
Name: rt, dtype: float64


,key,header=,pk,rt,pct_area,library_id,ref,cas,qual,area
0,FA03.D,1=,1.0,5.7877,2.0335,Methyl octanoate,17.0,000000-00-0,96.0,1489466.0
1,FA03.D,2=,2.0,7.3441,3.4015,Methyl decanoate,1.0,000000-00-0,98.0,2491449.0
2,FA03.D,3=,3.0,8.0364,1.7448,Methyl undecanoate,2.0,000000-00-0,98.0,1277982.0
3,FA03.D,4=,4.0,8.6715,3.9674,Methyl dodecanoate,3.0,000000-00-0,98.0,2905961.0
4,FA03.D,5=,5.0,9.2781,1.9607,Methyl tridecanoate,4.0,000000-00-0,99.0,1436154.0
5,FA03.D,6=,6.0,9.7930,1.8832,Methyl myristoleic acid,5.0,000000-00-0,99.0,1379404.0
6,FA03.D,7=,7.0,9.8503,4.7016,Methyl myristate,6.0,000000-00-0,99.0,3443757.0
7,FA03.D,8=,8.0,10.3595,1.9783,cis-10-Pentadecenoic acid methyl ester,7.0,000000-00-0,99.0,1449016.0
8,FA03.D,9=,9.0,10.4168,2.4864,Methyl myristate,6.0,000000-00-0,83.0,1821171.0
9,FA03.D,10=,10.0,10.9317,2.1450,Methyl palmitoleate,9.0,000000-00-0,99.0,1571132.0


In [48]:
libdf.reset_index().loc[libdf.reset_index()['key']=='FA03.D']

,key,header=,pk,rt,pct_area,library_id,ref,cas,qual
0,FA03.D,1=,1.0,5.7877,2.0335,Methyl octanoate,17.0,000000-00-0,96.0
1,FA03.D,2=,2.0,7.3441,3.4015,Methyl decanoate,1.0,000000-00-0,98.0
2,FA03.D,3=,3.0,8.0364,1.7448,Methyl undecanoate,2.0,000000-00-0,98.0
3,FA03.D,4=,4.0,8.6715,3.9674,Methyl dodecanoate,3.0,000000-00-0,98.0
4,FA03.D,5=,5.0,9.2781,1.9607,Methyl tridecanoate,4.0,000000-00-0,99.0
5,FA03.D,6=,6.0,9.7930,1.8832,Methyl myristoleic acid,5.0,000000-00-0,99.0
6,FA03.D,7=,7.0,9.8503,4.7016,Methyl myristate,6.0,000000-00-0,99.0
7,FA03.D,8=,8.0,10.3595,1.9783,cis-10-Pentadecenoic acid methyl ester,7.0,000000-00-0,99.0
8,FA03.D,9=,9.0,10.4168,2.4864,Methyl myristate,6.0,000000-00-0,83.0
9,FA03.D,10=,10.0,10.9317,2.1450,Methyl palmitoleate,9.0,000000-00-0,99.0


In [47]:
areadf.reset_index().loc[areadf.reset_index()['key']=='FA03.D']

,key,header=,peak,rt,first,max,last,pk_ty,height,area,pct_max,pct_total
9,FA03.D,1=,1.0,5.788,465.0,473.0,494.0,rBV3,257808.0,1489466.0,13.12,2.034
10,FA03.D,2=,2.0,7.344,733.0,745.0,812.0,rBV,964743.0,2491449.0,21.94,3.401
11,FA03.D,3=,3.0,8.036,859.0,866.0,904.0,rBV,608418.0,1277982.0,11.25,1.745
12,FA03.D,4=,4.0,8.672,970.0,977.0,1017.0,rBV,1929049.0,2905961.0,25.59,3.967
13,FA03.D,5=,5.0,9.278,1077.0,1083.0,1116.0,rBV,882521.0,1436154.0,12.65,1.961
14,FA03.D,6=,6.0,9.793,1168.0,1173.0,1179.0,rBV,1075433.0,1379404.0,12.15,1.883
15,FA03.D,7=,7.0,9.850,1179.0,1183.0,1220.0,rVB,2474155.0,3443757.0,30.32,4.702
16,FA03.D,8=,8.0,10.360,1267.0,1272.0,1278.0,rBV,1073796.0,1449016.0,12.76,1.978
17,FA03.D,9=,9.0,10.417,1278.0,1282.0,1315.0,rVB,1116633.0,1821171.0,16.04,2.486
18,FA03.D,10=,10.0,10.932,1363.0,1372.0,1386.0,rBV,1047443.0,1571132.0,13.83,2.145


This creates a df containing the concentrations, I assumed the starting concentration of the stock was 1M, to get from hanna. The general data input represented by __calc_conc_df__ will always be this format; however, we will need to have a different ways for the user to input this information

In [ ]:
cal_files = ['FA03.D','FA04.D','FA05.D']
cal_dilutions = {'FA03.D': 0.25,'FA04.D':0.50,'FA05.D':1.0}
standards = pd.DataFrame({'library_id':list(set(agi.results_lib.library_id)),'Conc_0':1})

for key,val in cal_dilutions.items():
    standards[key] = standards['Conc_0']*val
standards.drop('Conc_0',1,inplace=True)
standards.to_csv('CalibrationData.csv')
standards.head()

Calculate a linear fit to calibration data for each species

In [ ]:
list(standards.keys()[1:])

In [ ]:
def match_cal_conc(matcheddf, standards):
    '''
    this  function takes a dataframe which contains species matched to an area (matched_df) 
    and a calibration concentration dataframe and matches these two based on library_id
    '''
    standards_melted = pd.melt(standards, id_vars=['library_id'],value_vars=cal_files)
    standards_melted.columns = ['library_id','key','cal_conc']
    
    return_df = pd.merge(matcheddf.reset_index(),standards_melted,how='left',on=['library_id','key'])
    return return_df.dropna(subset=['cal_conc'])

def cal_curves(matched_cal_conc):
    '''
    this function takes a matched calibration concentration dataframe (matched_cal_conc)
    and does a linear regression and returns a dataframe of the library_ids with 
    linregress stats and the min/max areas which is the range for which the calibration
    curve can (should only) be used.
    
    this is only for cases where each molecule has a calibration curve i.e. tic and/or fid
    '''
    b = (matched_cal_conc.groupby('library_id')
                         .apply(lambda a: linregress(a.area,a.cal_conc))
                         .apply(pd.Series)
                         .reset_index())
    b.columns = ['library_id','slope','intercept','rvalue','pvalue','stderr']
    
    d = pd.DataFrame({'max':matched_cal_conc.groupby('library_id')['area'].max(),
                    'min':matched_cal_conc.groupby('library_id')['area'].min()}).reset_index()
    return pd.merge(b,d,on='library_id')

matched_cal_conc = match_cal_conc(matcheddf, standards)
calced_curves = cal_curves(matched_cal_conc)
calced_curves.head()


In [ ]:
def calc_conc(cal_curves, matcheddf):
    '''
    this  function takes a dataframe which contains species matched to an area (matchlib2area_all) 
    and a calibration curve dataframe (cal_curves_tic)
    
    it uses these values to calculate the concentration
    '''
    def conc_cal(x):
        conc = (x['area']*x['slope']+x['intercept'] if x['area']*x['slope']+x['intercept']>0 else np.nan)
        return conc
    
    # calculate concentration of species
    matcheddf = matcheddf.reset_index()
    return_df = pd.merge(matcheddf,cal_curves,on='library_id',how='outer')
    return_df['conc'] = return_df.apply(conc_cal,axis=1)
    return_df.drop(['rvalue','pvalue','stderr'],1,inplace=True)

    #calculate concentration percentage 
    totals_c = pd.DataFrame({'totals_c':(return_df.groupby('key')['conc']
                                            .apply(np.sum,axis=0))}).reset_index()                     
    return_df = return_df.merge(totals_c, on=['key'])
    return_df['conc%']=return_df['conc']/return_df['totals_c']
    return_df.drop(['totals_c'],1,inplace=True)
    
    #calculate area percentage
    totals_a = pd.DataFrame({'totals_a':(return_df.groupby('key')['area']
                                            .apply(np.sum,axis=0))}).reset_index()                     
    return_df = return_df.merge(totals_a, on=['key'])
    return_df['area%']=return_df['area']/return_df['totals_a']
    return_df.drop(['totals_a'],1,inplace=True)
    
    return return_df.set_index('key')

calced_conc = calc_conc(calced_curves, matcheddf)
calced_conc

In [ ]:
from bokeh.charts import Bar, show, output_notebook
from bokeh.layouts import row
output_notebook()

def calc_conc_pivot(calced_conc,standards = pd.DataFrame()):
    calced_conc = calced_conc.reset_index()
    standard_keys = list(standards.keys()[1:])
    
    #Data for Concetration Stacked Concentraction Percent
    unknowns_per_df = calced_conc[-calced_conc['key'].isin(standard_keys)]
    unknowns_per_pvt = pd.pivot_table(unknowns_per_df,
                               index='key',
                               columns='library_id',
                               values='conc%')
    #Data for Concentration Bar
    unknowns_df = calced_conc[-calced_conc['key'].isin(standard_keys)]
    unknowns_pvt = pd.pivot_table(unknowns_df,
                               index='key',
                               columns='library_id',
                               values='conc')
    return {'percent':unknowns_per_pvt,'concentration':unknowns_pvt}

def plot_gcdata(calced_conc,standards = pd.DataFrame()):
    calced_conc = calced_conc.reset_index()
    standard_keys = list(standards.keys()[1:])
    
    #Data for Concetration Stacked Concentraction Percent
    unknowns_per_df = calced_conc[-calced_conc['key'].isin(standard_keys)]

    #Data for Concentration Bar
    unknowns_df = calced_conc[-calced_conc['key'].isin(standard_keys)]

    #Plot stacked concentration percentage bar
    bar_per = Bar(unknowns_per_df,'key',values = 'conc%',agg='sum',stack='library_id')

    #Plot concentration bar
    bar_conc = Bar(unknowns_per_df,'key',values = 'conc',group = 'library_id')
    
    #show plots in row
    show(row(bar_per,bar_conc))    
    return 

plot_gcdata(calced_conc,standards)
calc_conc_pivot(calced_conc,standards)['percent']

Fuctions which still need to be built

In [ ]:


def make_plot_percentarea():
    pass

def make_plot_concarea():
    pass

def make_plot_massarea():
    pass

def calc_mass():
    pass